In [7]:
!pip install -qU langchain-core==0.2.27 langchain-community langchain-openai

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-huggingface 0.1.0 requires langchain-core<0.4,>=0.3.0, but you have langchain-core 0.2.27 which is incompatible.


In [8]:
!pip install -qU pymupdf ragas

In [9]:
from langchain_community.document_loaders import PyMuPDFLoader
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

import os
import getpass
from uuid import uuid4

/opt/anaconda3/envs/ai-playground/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### set up env

In [ ]:

# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

In [ ]:

# os.environ["LANGCHAIN_PROJECT"] = f"AIM-MIDTERM - SDG - {uuid4().hex[0:8]}"

In [10]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

### Generate testset

In [11]:

documents = PyMuPDFLoader(file_path="data/Blueprint-for-an-AI-Bill-of-Rights.pdf").load()

generator_llm = ChatOpenAI(model="gpt-3.5-turbo")
critic_llm = ChatOpenAI(model="gpt-4o-mini")
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

distributions = {
    simple: 0.5,
    multi_context: 0.4,
    reasoning: 0.1
}

In [12]:
testset = generator.generate_with_langchain_docs(documents, 10, distributions, with_debugging_logs=True)
testset_df = testset.to_pandas()
testset_df

Filename and doc_id are the same for all nodes.                   
Generating:   0%|          | 0/10 [00:00<?, ?it/s][ragas.testset.filters.DEBUG] context scoring: {'clarity': 2, 'depth': 3, 'structure': 2, 'relevance': 3, 'score': 2.5}
[ragas.testset.evolutions.DEBUG] keyphrases in merged node: ['Technology use in social welfare', 'Healthcare system', 'Health disparities', 'Racial biases in medicine', 'Community input']
[ragas.testset.filters.DEBUG] context scoring: {'clarity': 3, 'depth': 3, 'structure': 3, 'relevance': 3, 'score': 3.0}
[ragas.testset.evolutions.DEBUG] keyphrases in merged node: ['Human alternatives', 'Opt out', 'Automated systems', 'Fallback process', 'Human consideration']
[ragas.testset.filters.DEBUG] context scoring: {'clarity': 3, 'depth': 3, 'structure': 3, 'relevance': 3, 'score': 3.0}
[ragas.testset.evolutions.DEBUG] keyphrases in merged node: ['Data privacy', 'Automated systems', 'Surveillance oversight', 'Algorithmic discrimination', 'Consent practices']
[r

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What benefits have been publicly described by ...,[ \nENDNOTES\n12. Expectations about reporting...,The benefits of 'traffic calming' measures hav...,simple,[{'source': 'data/Blueprint-for-an-AI-Bill-of-...,True
1,How does the Blueprint for an AI Bill of Right...,[SECTION TITLE\n \n \n \n \n \n \nApplying The...,The Blueprint for an AI Bill of Rights aims to...,simple,[{'source': 'data/Blueprint-for-an-AI-Bill-of-...,True
2,What expectations should be met by automated s...,[ \n \n \n \n \n \nDATA PRIVACY \nWHAT SHOULD ...,The expectations for automated systems in term...,simple,[{'source': 'data/Blueprint-for-an-AI-Bill-of-...,True
3,What stakeholders were involved in providing i...,[APPENDIX\n• OSTP conducted meetings with a va...,Stakeholders involved in providing ideas relat...,simple,[{'source': 'data/Blueprint-for-an-AI-Bill-of-...,True
4,What stakeholders were involved in the meeting...,[APPENDIX\n• OSTP conducted meetings with a va...,Stakeholders involved in the meetings conducte...,simple,[{'source': 'data/Blueprint-for-an-AI-Bill-of-...,True
5,What advantages does The Blueprint for an AI B...,[ \n \n \nApplying The Blueprint for an AI Bil...,The answer to given question is not present in...,multi_context,[{'source': 'data/Blueprint-for-an-AI-Bill-of-...,True
6,How can equity be ensured in automated system ...,[ \n \n \n \n \n \n \nWHAT SHOULD BE EXPECTED ...,Equity in automated system design can be ensur...,multi_context,[{'source': 'data/Blueprint-for-an-AI-Bill-of-...,True
7,What criteria should automated systems in sens...,"[ \n \n \n \n \n \n \nHUMAN ALTERNATIVES, \nCO...",Automated systems used within sensitive domain...,multi_context,[{'source': 'data/Blueprint-for-an-AI-Bill-of-...,True
8,What privacy measures should automated systems...,[ \n \n \n \n \n \nDATA PRIVACY \nWHAT SHOULD ...,Automated systems should include privacy measu...,multi_context,[{'source': 'data/Blueprint-for-an-AI-Bill-of-...,True
9,"How does AI ""nudification"" tech affect women's...",[ \n \n \n \nSAFE AND EFFECTIVE \nSYSTEMS \nW...,"AI-enabled ""nudification"" technology creates i...",reasoning,[{'source': 'data/Blueprint-for-an-AI-Bill-of-...,True


In [13]:
testset_df.to_csv('data/testset.csv')